The purpose of this notebook is to voxelize and save all 20 of the simulated events in the /tutorial_files/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root file.

Modified from ~/icdl/notebooks/tutorials/tutorial_make_truthlabeled_spacepoints_and_voxels.ipynb tutorial written by Taritree

# Import useful packages

In [1]:
import os,sys
import plotly as pl
import plotly.graph_objects as go
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
# on trex, for some reason, need to load ROOT in separate cell before loading icdl modules
import ROOT as rt
from ROOT import std

Welcome to JupyROOT 6.24/06


In [3]:
from larlite import larlite
from larlite import larutil
from larcv import larcv
from ublarcvapp import ublarcvapp
from larflow import larflow
import lardly

# Load the data file
Below we specify the location of the file we want to use with
* simulated data from the detector (the three wire plane images)
* truth information we can use to label generated spacepoints

In [4]:
# Specify location of the file we want to open
inputfile = "/tutorial_files/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root"

# we need the name of the TTree that stores our images
WIRE_ADC = "wire"

# Use ROOT to open the file
rfile = rt.TFile(inputfile,"open")
# rfile.ls() # To show the contents of the file

In [5]:
ioll = larlite.storage_manager( larlite.storage_manager.kREAD )
ioll.add_in_filename(  inputfile )
ioll.open()
iolcv = larcv.IOManager( larcv.IOManager.kREAD, "larcv", larcv.IOManager.kTickBackward ) # we use kTickBackward to make it work
iolcv.add_in_file( inputfile )
iolcv.reverse_all_products() # Since we read in backwards
iolcv.initialize()

NENTRIES = iolcv.get_n_entries()
print("Number of entries in file: ",NENTRIES)
assert(NENTRIES == 20)

Number of entries in file:  20
    [NORMAL]  <open> Opening a file in READ mode: /tutorial_files/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root
    [NORMAL]  <larcv::prepare_input> Opening a file in READ mode: /tutorial_files/dlmerged_larflowtruth_mcc9_v13_bnbnue_corsika_run00001_subrun00001.root
    [NORMAL]  <larcv::initialize> Prepared input with 20 entries...


Error in <TChain::SetBranchAddress>: The pointer type given (larlite::event_base) does not correspond to the class needed (larcv::EventChStatus) by the branch: chstatus_wire_branch
Error in <TChain::SetBranchAddress>: The pointer type given (larlite::event_base) does not correspond to the class needed (larcv::EventChStatus) by the branch: chstatus_wiremc_branch


# Set the Detector Geometry

To calculate wire intersection points, we need the geometry of the detector.

To speed up spacepoint proposal generation, we actually do not calculate wire-intersections at runtime.
Instead we pre-calculate which wires intersect with which and store these in detector-specific "overlap matrix" files.

These overlap files are generated by code in `icdl/spacepointgen`.

However, DON'T remake these. You can ask someone for these files.

In [6]:
DETECTOR = "uboone"

# choices: "icarus", "uboone", "sbnd"
overlap_folder = "/tutorial_files/"

if DETECTOR == "icarus":
    detid = larlite.geo.kICARUS
    overlap_matrix_file = overlap_folder+"/output_icarus_wireoverlap_matrices.root"
elif DETECTOR == "uboone":
    detid = larlite.geo.kMicroBooNE
    overlap_matrix_file = overlap_folder+"/output_microboone_wireoverlap_matrices.root"
elif DETECTOR == "sbnd":
    detid = larlite.geo.kSBND
    overlap_matrix_file = "does_not_exist_yet.root"
    
if not os.path.exists(overlap_matrix_file):
    print("WARNING: Could not find overlap matrices")
else:
    print("Found overlap matrix file")
    
larutil.LArUtilConfig.SetDetector(detid)

Found overlap matrix file


True

# Prepare the output file & algorithm objects

In [7]:
# Safeguard
try:
    outfile.Close()
except:
    pass

# We are going to define a ROOT file for the algorithms to write out to
outfilename = "20spacepointTutorialData.root"
outfile = rt.TFile(outfilename,"recreate")
triptree = rt.TTree("larmatchtriplet","LArMatch triplets") # a new tree to store triplet data

# triplet proposal maker      
tripletmaker = larflow.prep.PrepMatchTriplets()
tripletmaker.set_wireoverlap_filepath( overlap_matrix_file  )
ev_tripdata = std.vector("larflow::prep::MatchTriplets")() # container to store triplet maker output
triptree.Branch("triplet_v",ev_tripdata) # add the container to the triplet tree, triptree

# Keypoint maker
kpana = larflow.keypoints.PrepKeypointData()
#kpana.set_verbosity( larcv.msg.kDEBUG )
kpana.setADCimageTreeName( WIRE_ADC )
outfile.cd()
kpana.defineAnaTree()

# ssnet label data: provides particle label for each spacepoint                                                                                                                                                                          
ssnet = larflow.prep.PrepSSNetTriplet()
outfile.cd()
ssnet.defineAnaTree()

# We make spacepoints from 2D information, so there are some mistakes we fix using various methods
truthfixer = larflow.prep.TripletTruthFixer()

# Voxelizer:
# The Voxelizer Class
# See larflow/larflow/Voxelizer/VoxelizeTriplets.h for more info
# 
voxelizer = larflow.voxelizer.VoxelizeTriplets()


    [NORMAL]  <ReadTree> Loading geo data for DetID=2 ...
 file=/home/nstieg01/icdl/larlite/larlite//LArUtil/dat/microboone_larlite_geodata.root 
    [NORMAL]  <LoadData>  max channel numbers: 8255
    [NORMAL]  <LoadData>  geometry loaded successfully.
    [NORMAL]  <LoadData> Loading data for DetectorProperties...
     file=/home/nstieg01/icdl/larlite/larlite//LArUtil/dat/larutil_microboone.root 
old fXTicksOffsets branch used for MicroBooNE
SimChannelVoxelizer: tickmin=2400 tickmax=8448  dtick=1
    [NORMAL]  <LoadData> Loading data for LArProperties...
     file=/home/nstieg01/icdl/larlite/larlite//LArUtil/dat/larutil_microboone.root 
old fEfield branch used for MicroBooNE
SimChannelVoxelizer: tickmin=2400 tickmax=8448  dtick=6
[VoxelizeTriplets::_define_voxels.L80] 
 CRYO[0] TPC[0] 
 NVOXELS: (1009,781,3457)
 VOXEL LENGTH: (6,0.3,0.3)
 BOUNDS: [2400,8454] [-116.03,118.27] [0,1037.1] 


# Run algorithms on x <= 20 events, creating spacepoints

In [8]:
# Loop over 20 entries in original file to turn them all into spacepoints
# for i in range(0, 20):
# NUM_TO_DO = 1
# for i in range (0, NUM_TO_DO):
for i in range (0, 1):

    print(f"Now running on entry: {i}")
    # Now we load an event
    ENTRY_NUM = i
    
    ioll.go_to(ENTRY_NUM)
    
    iolcv.read_entry(ENTRY_NUM)
    
    # Run the algorithms


    # mcpg = ublarcvapp.mctools.MCPixelPGraph()
    # mcpg.buildgraphonly( ioll )
    # mcpg.printGraph(0,False)
    # sys.stdout.flush()
    outfile.cd()
    
    # Test
    # tripletmaker = larflow.prep.PrepMatchTriplets()
    # tripletmaker.set_wireoverlap_filepath( overlap_matrix_file  )
    # ev_tripdata = std.vector("larflow::prep::MatchTriplets")() # container to store triplet maker output
    # triptree.Branch("triplet_v",ev_tripdata) # add the container to the triplet tree, triptree
    
    # make triplet proposals: function valid for simulation or real data                                                                                                                                                                  
    tripletmaker.process( iolcv, WIRE_ADC, WIRE_ADC, 10.0, True )

    # make good/bad triplet ground truth                                                                                                                                                      
    tripletmaker.process_truth_labels( iolcv, ioll, WIRE_ADC )

    # fix up some labels
    truthfixer.calc_reassignments( tripletmaker, iolcv, ioll )

    # # make keypoint score ground truth                                                                                                                                                        
    kpana.process( iolcv, ioll )
    kpana.make_proposal_labels( tripletmaker )
    kpana.fillAnaTree()

    # # make ssnet ground truth                                                                                                                                                                 
    ssnet.make_ssnet_labels( iolcv, ioll, tripletmaker )
    
    # i = 0
    for imatch in range(tripletmaker._match_triplet_v.size()):
        ev_tripdata.push_back( tripletmaker._match_triplet_v.at(imatch) )
    #     i += 1
    
    # print(i)
    
    print(ev_tripdata.size())
    triptree.Fill()
    
    # iolcv.clear()
    # tripletmaker.clear()
    

outfile.cd()
# write the data to file and then close the file
kpana.writeAnaTree()
ssnet.writeAnaTree()
triptree.Write()
outfile.Close()

Now running on entry: 0
1
    [NORMAL]  <PrepMatchTriplets::process> Have bad channel info make badchannel images
    [NORMAL]  <PrepMatchTriplets::process> Store adc_v image[0] for (plane,tpc,cryo)=(0,0,0)
    [NORMAL]  <PrepMatchTriplets::process> Store adc_v image[1] for (plane,tpc,cryo)=(1,0,0)
    [NORMAL]  <PrepMatchTriplets::process> Store adc_v image[2] for (plane,tpc,cryo)=(2,0,0)
[FlowTriples] plane[0] has 8533 (above threshold) pixels
[FlowTriples] plane[1] has 11321 (above threshold) pixels
[FlowTriples] plane[2] has 7506 (above threshold) pixels
    [NORMAL]  <PrepMatchTriplets::process_tpc_v2> made total of 331477 nrepeated=91718 unique index triplets. time elapsed=4.80681
    [NORMAL]  <PrepMatchTriplets::make_truth_vector>  (cryo,tpcid)=0,0) num larflow truth images=6
    [NORMAL]  <PrepMatchTriplets::make_truth_vector> === (cryoid,tpcid)=(0,0) =======
    [NORMAL]  <PrepMatchTriplets::make_truth_vector>  number of triplets: 331477
    [NORMAL]  <PrepMatchTriplets::make

# Load x <= 20 entries & voxelize them

In [9]:
# Our plotting tools -- and the deep learning frameworks work with numpy arrays.
# We have written C++ functions that output the spacepoints and labels we've generated into
#  such numpy arrays
from ctypes import c_int
from larflow import larflow
larflow.keypoints.LoaderKeypointData

f_v = std.vector("std::string")()
f_v.push_back(outfilename)
kploader = larflow.keypoints.LoaderKeypointData( f_v )
kploader.set_verbosity( larcv.msg.kDEBUG )
kploader.exclude_false_triplets( False )
kploader.GetEntries()

1

Error in <TChain::LoadTree>: Cannot find tree with name LArbysMCTree in file 20spacepointTutorialData.root


In [10]:
# first make a VoxelSet, this is a container that holds the description of the active voxels 
from larcv import larcv

# translation from how we label particle types to how lartpc_mlreco3d labeled particles
# mlreco particle labels found in lartpc_mlreco3d/mlreco/utils/groups.py
# larflow particle labels found in larflow/larflow/PrepFlowMatchData/PrepSSNetTriplet.h
larcv2mlreco = {0:5, # bg -> ghost
                1:1, # electron -> electron
                2:0, # photon -> photon
                3:2, # muon -> muon 
                4:3, # pion -> pion
                5:4, # proton -> proton
                6:3, # other (usually mesons) -> pion
               }

In [11]:
# create an output file
voxel_outfile_name = "20_voxelized_tutorial_data.root"
out_larcv = larcv.IOManager( larcv.IOManager.kWRITE, "voxeldata" )
out_larcv.set_out_file( voxel_outfile_name )
out_larcv.initialize()

for i in range(0, 1):

    print(f"Loading entry {i}")
    
#     kploader = larflow.keypoints.LoaderKeypointData( f_v )
#     kploader.set_verbosity( larcv.msg.kDEBUG )
#     kploader.exclude_false_triplets( False )
    
    kploader.load_entry(i)

    # the original image data in sparse2D array form
    tripletmaker = kploader.triplet_v.at(0).setShuffleWhenSampling( False )
    # print(tripletmaker)

    # 2d images                                                                                                                                                                               
    wireimg_dict = {}
    for p in range(3):
        wireimg = kploader.triplet_v.at(0).make_sparse_image( p )
        wireimg_coord = wireimg[:,:2].astype(np.long)
        wireimg_feat  = wireimg[:,2]
        wireimg_dict["wireimg_coord%d"%(p)] = wireimg_coord
        wireimg_dict["wireimg_feat%d"%(p)] = wireimg_feat


    tripdata = kploader.triplet_v.at(0).get_all_triplet_data(True)
    spandata = kploader.triplet_v.at(0).get_matchspan_array().astype(np.float32)
    spacepoints = kploader.triplet_v.at(0).make_spacepoint_charge_array()

    nfilled = c_int(0)
    ntriplets = tripdata.shape[0]

    kplabel_sigma = 5.0
    TPCID = 0
    CRYOID = 0
    data = kploader.sample_data( ntriplets, nfilled, True, kplabel_sigma, TPCID, CRYOID )
    sys.stdout.flush()

    #data.update(kpdata)
    data.update(spacepoints)
    data["truespan_t"] = spandata

    print("KeypointLoader returned dictionary with event data. KEYS: ")
    print(data.keys())

    voxeldata = voxelizer.get_full_voxel_labelset_dict( kploader )
    # this returns a python list of dictionaries. 
    # the latter contains voxel data and labels for each TPC in the detector.
    # (for detectors with more than one TPC. MicroBooNE has just one.)

    # pick out one TPC's worth of data
    vdata = voxeldata[0]
#     print(vdata.keys())
#     print("voxel coordinate tensor: ",vdata['voxcoord'].shape)
#     print("voxel feature tensor: ",vdata['voxfeat'].shape)
#     print("voxel ssnet  label tensor: ",vdata['ssnet_labels'].shape)
#     print("voxel true/ghost labels: ",vdata['voxlabel'].shape)
#     print("voxel array origin: ",vdata['voxorigin'].shape)

#     print("unique ssnet labels in tensor: ",np.unique(vdata['ssnet_labels']))
    # get the voxel arrays we need
    voxcoord = vdata['voxcoord']
    voxfeat  = vdata['voxfeat']
    voxssnet = vdata['ssnet_labels']
    # each voxel needs an ID number. We use the unrolled array position
    # to calculate this, we need array strides for each dimension
    strides = [ voxelizer.get_nvoxels()[0]*voxelizer.get_nvoxels()[1],
                voxelizer.get_nvoxels()[2],
                1]

    
    vset_uplane = larcv.VoxelSet() # save y-plane charge
    vset_vplane = larcv.VoxelSet() # save y-plane charge
    vset_yplane = larcv.VoxelSet() # save y-plane charge
    vset_ssnet  = larcv.VoxelSet() # save particle labels

    # some voxelset functions work best when voxels are added in sorted order (by ID number)
    # so we do that first
    idnums = []
    id2index = {}
    for n in range( voxcoord.shape[0] ):
        # create a voxel: this consists of an ID and value.
        # the ID is the array index
        idnum = int( voxcoord[n,0]*strides[0] + voxcoord[n,1]*strides[1] + voxcoord[n,2]*strides[2] )
        idnums.append(idnum)
        id2index[idnum] = n

    # sort the different idnumbers
    idnums.sort()
    for idnum in idnums:
        # this format only allows us one feature: here filling with Y-plane values 
        vset_uplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],0] ) )
        vset_vplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],1] ) )
        vset_yplane.add( larcv.Voxel( idnum, voxfeat[id2index[idnum],2] ) )
        ssnetlabel = int(voxssnet[id2index[idnum]])
        # there needs to be come translation between our label definitions to lartpc_mlreco3d label definitions

        mlrecolabel = larcv2mlreco[ssnetlabel]
        vset_ssnet.add( larcv.Voxel(idnum, float(mlrecolabel) ))

    # now we need the meta, that helps us go from array index to position in detector space
    meta3d = larcv.Voxel3DMeta()
    """
    inline void set(double xmin, double ymin, double zmin,
                        double xmax, double ymax, double zmax,
                        size_t xnum,size_t ynum,size_t znum,
                        DistanceUnit_t unit=kUnitCM)
    """
    meta3d.set( voxelizer.get_origin()[0], voxelizer.get_origin()[1], voxelizer.get_origin()[2],
                voxelizer.get_origin()[0]+voxelizer.get_dim_len()[0],
                voxelizer.get_origin()[1]+voxelizer.get_dim_len()[1],
                voxelizer.get_origin()[2]+voxelizer.get_dim_len()[2],
                voxelizer.get_nvoxels()[0], voxelizer.get_nvoxels()[1], voxelizer.get_nvoxels()[2] )


    # we need an event container for the tensor. 
    # we get one from the IOManager so that is all setup to be saved to the root tree
    # note the name of the containers is based on the lartpc_mlreco3d config 'config_uresnet_ppn.cfg'
    #  * for charge data: "pcluster"
    #  * for ssnet labels: "pcluster_semantic"
    event_vox3d_uplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_uplane" )
    event_vox3d_vplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_vplane" )
    event_vox3d_yplane = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_yplane" )
    event_vox3d_ssnet  = out_larcv.get_data( larcv.kProductSparseTensor3D, "pcluster_semantic" )
    # add the data
    event_vox3d_uplane.set( vset_uplane, meta3d )
    event_vox3d_vplane.set( vset_vplane, meta3d )
    event_vox3d_yplane.set( vset_yplane, meta3d )
    event_vox3d_ssnet.set(  vset_ssnet,  meta3d )

    # save the event to file
    out_larcv.set_id(  0, 0, i) # set an arbitrary (run,subrun,event) index
    out_larcv.save_entry()
    
    # tripletmaker.clear()
        

# finalize the file (properly close file)
out_larcv.finalize()

Loading entry 0
KeypointLoader returned dictionary with event data. KEYS: 
dict_keys(['matchtriplet', 'match_weight', 'positive_indices', 'ssnet_label', 'ssnet_top_weight', 'ssnet_class_weight', 'kplabel', 'kplabel_weight', 'spacepoint_t', 'truetriplet_t', 'segment_t', 'truespan_t'])


AttributeError: 'VoxelizeTriplets' object has no attribute 'get_nvoxels'

      [INFO]  <LoaderKeypointData::load_entry::L126> Loaded trees (ttriplet,tssnet,tkeypoint)
[MatchTriplets::make_spacepoint_charge_array] number of true points: 70247 of 331477
     [DEBUG]  <LoaderKeypointData::sample_data> LoaderKeypointData.cxx::L179 make triplets (cryo,tpcid)=(0,0)
     [DEBUG]  <LoaderKeypointData::sample_data> LoaderKeypointData.cxx::L214  npos=70247 nneg=261230
     [DEBUG]  <LoaderKeypointData::sample_data> LoaderKeypointData.cxx::L242 call make_ssnet_arrays
     [DEBUG]  <LoaderKeypointData::make_ssnet_arrays> LoaderKeypointData.cxx::L333 pos_match_index=70247 withtruth=1 num_max_samples=331477
     [DEBUG]  <LoaderKeypointData::make_ssnet_arrays> LoaderKeypointData.cxx::L356 make class labels and topological weight arrays. nelems=331477
     [DEBUG]  <LoaderKeypointData::make_ssnet_arrays> LoaderKeypointData.cxx::L401 make class balancing weights
     [DEBUG]  <LoaderKeypointData::make_ssnet_arrays> LoaderKeypointData.cxx::L429 Num bad labels: 0


# Set up for visualization

In [ ]:
# We generate the TPC boundary lines for plotting and define useful defaults for our figures

import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

plot_layout = {
    "title": "",
    "height":800,
    "margin": {"t": 0, "b": 0, "l": 0, "r": 0},
    "font": {"size": 12, "color": "black"},
    "showlegend": False,
    "plot_bgcolor": "white",
    "paper_bgcolor": "white",
    "scene": {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": 3, "y": 2, "z": 2},
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    },
}

# PARTICLE LABEL COLORS
# from larcv/core/DataFormat/DataFormatTypes.h
#     kROIUnknown=0, ///< LArbys
#     kROICosmic,    ///< Cosmics
#     kROIBNB,       ///< BNB
#     kROIEminus,    ///< Electron
#     kROIGamma,     ///< Gamma
#     kROIPizero,    ///< Pi0
#     kROIMuminus,   ///< Muon
#     kROIKminus,    ///< Kaon
#     kROIPiminus,   ///< Charged Pion
#     kROIProton,    ///< Proton
#     kROITypeMax    ///< enum element counter
ssnetcolor = {0:np.array((0,0,0)),     # kROIUnknown                                                                                                                                                   
              1:np.array((255,0,0)),   # kROICosmic (not used)                                                                                                                                       
              2:np.array((0,255,0)),   # kROIBNB (not used)                                                                                                                             
              3:np.array((0,0,255)),   # kROIEminus (e-/e+)                                                                                                                                              
              4:np.array((255,0,255)), # kROIGamma                                                                                                                                                 
              5:np.array((0,255,255)), # kROIPizero                                                                                                                                            
              6:np.array((255,255,0)), # kROImuminus (mu-/mu+)
              7:np.array((123,300,10)),# kROIKminus (k+/k-)
              8:np.array((204,204,255)), # kROIPiminus (pi+/pi-)
              9:np.array((255, 165, 0))} # kProton

kpcolors = {0:np.array((255,0,0)), # nu
            1:np.array((0,255,0)), # track-start
            2:np.array((0,0,255)), # track-end
            3:np.array((255,255,0)), # shower
            4:np.array((0,255,255)), # michel
            5:np.array((255,0,255))} # delt